# References and Copies in Python

This notebook explains **references vs copies** in Python in a way that does not require prior knowledge of memory internals.


##  The most important rule

In Python, variables do **not** store the object itself. They store a **reference** (a label/address) to an object.

## Referencing
Consider the following case:


In [1]:
a = [1, 2, 3]
b = a
b[2] = 11

print("list a:", a)
print("list b:", b)

list a: [1, 2, 11]
list b: [1, 2, 11]


We created a new variable `b` and assigned the value of list `a` to it, after modifying the value in list `b`, what would be the result of list `a` and `b`?

Consider a second case:



In [2]:
a = [1, 2, 3]

def func(input_list):
    input_list[2] = 11
    return input_list

b = func(a)
print("list a:", a)
print("list b:", b)

list a: [1, 2, 11]
list b: [1, 2, 11]


In this example, list `a` is passed in as an input to our function, and we modified the input inside the function, what would happen to `a` ?

In both the examples above, you may have guessed, the result of a would be modified un-intentionally.

This is because in both the scenarios above, the new variable that we created actually binds to the same memory address of `a`, which means although it seems that we created a new variable `b`, it does not take actual memory space and the new variable was a reference to list `a`.

An illustrative diagram would be this:


![Referencing in Python](images/refrencing_python.webp)



## 2) Stack vs Heap (simple mental model)

You do not need low-level details to program Python, but a simple model helps.

### Stack (simple view)
- Used for **function calls** and **local names** (variables inside functions).
- When a function ends, its local names go away.

### Heap (simple view)
- Where **objects live** (lists, dicts, strings, arrays, class instances, etc.).
- Objects can live longer than a single function call.

### Key takeaway
**Names are temporary labels. Objects are the real data.**


## 3) Reference vs Copy

- **Reference:** two variables can point to the *same* object.
- **Copy:** you create a *new* object with the same (or similar) content.


## 4) Mutable vs Immutable

- **Immutable:** cannot be changed in place (e.g., `int`, `str`, `tuple`).
- **Mutable:** can be changed in place (e.g., `list`, `dict`, many class instances).


## 5) Experiment: assignment shares references (mutable case)

In [ ]:
a = [1, 2, 3]
b = a

b[0] = 100

print('a =', a)
print('b =', b)
print('a is b:', a is b)


**Notice:** `a is b` is `True`, meaning both names refer to the same object.


## 6) Experiment: immutable objects behave differently

In [ ]:
x = 10
y = x

y = y + 1

print('x =', x)
print('y =', y)
print('x is y:', x is y)


**Notice:** integers are immutable, so `y = y + 1` creates a new object and rebinds `y`.


## 7) How to make a copy

Python will not copy automatically on assignment. You must copy explicitly.

- **Shallow copy:** copies only the outer container; nested objects can remain shared.
- **Deep copy:** recursively copies nested objects so nothing is shared.


### Shallow copy example

In [ ]:
import copy

original = [1, 2, [3, 4]]
shallow = copy.copy(original)

original[2].append(5)

print('original =', original)
print('shallow  =', shallow)
print('outer shared? ', original is shallow)
print('inner shared? ', original[2] is shallow[2])


**Explanation:** shallow copy copied the outer list, but the inner list is still shared.


### Deep copy example

In [ ]:
import copy

original = [1, 2, [3, 4]]
deep = copy.deepcopy(original)

original[2].append(5)

print('original =', original)
print('deep     =', deep)
print('outer shared? ', original is deep)
print('inner shared? ', original[2] is deep[2])


## 8) References inside classes

Attributes are references too.


In [ ]:
class Box:
    def __init__(self, data):
        self.data = data

box = Box([1, 2, 3])
external_ref = box.data

external_ref[0] = 999

print('box.data     =', box.data)
print('external_ref =', external_ref)
print('same object? =', box.data is external_ref)


## 9) Connection to optimizers

In gradient descent, a parameter $p$ is updated by:

$p \leftarrow p - \eta g$

Optimizers work because they receive **references** to parameter objects, not copies.


### Tiny optimizer demo

In [ ]:
class ToyLayer:
    def __init__(self, W, dW):
        self.W = W
        self.dW = dW

    def params(self):
        return [self.W]

    def grads(self):
        return [self.dW]

class ToySGD:
    def __init__(self, lr=0.1):
        self.lr = lr

    def step(self, model):
        for p, g in zip(model.params(), model.grads()):
            p[0] = p[0] - self.lr * g[0]

layer = ToyLayer(W=[10, 20], dW=[1, 1])
opt = ToySGD(lr=0.1)

print('Before:', layer.W)
opt.step(layer)
print('After :', layer.W)


## 10) Checklist

1. Assume assignment shares references unless you explicitly copy.
2. Mutable objects can change in place (and other references will see it).
3. Use shallow copies for simple containers.
4. Use deep copies when nested objects must be independent.
5. Optimizers rely on references to update real parameters.
